# Inspect decaying trajectories

In [1]:
import numpy as np
import os
import pickle

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

Select folder where decay trajectories (.pkl files) are located and location of rest file (containing equilibrium position)

In [8]:
locs = [
    "origin",
    "north",
    "west",
    "south",
    "east",
    "northwest",
    "southwest",
    "southeast",
    "northeast"
]
decayData_dir = [f"/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic/{loc}/decay" for loc in locs]

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = os.path.join(robot_dir, 'rest_qv.pkl')

output_node = 'all' # 1354 # 51 # 
t_in = 1
t_out = 4

Load all trajectories

In [16]:
Data = {}
Data['oData'] = utils.import_pos_data(decayData_dir, rest_file, output_node, t_in, t_out, return_velocity=True)

In [17]:
print(Data['oData'][0][0].shape)
print(Data['oData'][0][1].shape)
nTraj = len(Data['oData'])
print(f"nTraj: {nTraj}")

(400,)
(6, 400)
nTraj: 2


In [18]:
with open(os.path.join(decayData_dir, "inputs.pkl"), "rb") as f:
    inputs = pickle.load(f)
inputs

[array([800.,   0., 800., 800., 800., 800.,   0., 800.]),
 array([  0., 800., 800.,   0., 800., 800.,   0.,   0.])]

Check that all the initial conditions differ

In [8]:
# equal_trajectories = []
# for i in range(len(Data['oData'])):
#     equal_count = 0
#     for j in range(i, len(Data['oData'])):
#         if i == j:
#             continue
#         equal = np.allclose(Data['oData'][i][1][:, 0], Data['oData'][j][1][:, 0], rtol=0, atol=0.0000001)
#         if equal:
#             equal_count += 1
#             equal_trajectories.append((i, j))
#     if equal_count > 0:
#         print(f"something wrong with trajectory {i}", equal_count)
# print(equal_trajectories)

Plot decay data in (x, y, z)

In [19]:
# outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = []
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)
# plot velocities
plot.traj_xyz(Data,
             xyz_idx=[('oData', 3), ('oData', 4), ('oData', 5)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)

Determine equilibrium position under given pre-tensioning

In [20]:
# print(Data['oData'][0][1][:3, -1])
z_eq = np.mean([Data['oData'][i][1][:3, -1] for i in range(nTraj)], axis=0)
qv_eq = np.hstack((z_eq, np.zeros(3)))
print(qv_eq)

[-34.89298671  35.84711563 -10.60749796   0.           0.
   0.        ]


Save equilibrium position into info.yaml dict

In [36]:
import yaml

with open(os.path.join(decayData_dir, "info.yaml"), "r") as f:
    info = yaml.safe_load(f)
info['qv_eq'] = qv_eq.astype(float).tolist()
with open(os.path.join(decayData_dir, "info.yaml"), "w") as f:
    yaml.dump(info, f)

Save data to .csv (one file per trajectory)

In [9]:
from tqdm.auto import tqdm
from os import mkdir
from os.path import exists

for i, dir in enumerate(tqdm(decayData_dir)):
    loc = locs[i]
    roshan_dir = f"{dir}/roshan_{loc}"
    if not exists(roshan_dir):
        mkdir(roshan_dir)
    oData = utils.import_pos_data(dir, rest_file, output_node, t_in, t_out, return_velocity=True)
    for i in tqdm(range(len(oData)), leave=False):
        if i == 0:
            # save time array only once, as it is the same for all trajectories
            np.savetxt(f"{roshan_dir}/t_{loc}.csv", oData[i][0], fmt='%.5e')
        np.savetxt(f"{roshan_dir}/decay_trajectory_{loc}_{i:02d}.csv", oData[i][1], fmt='%.5e')

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [11]:
import pickle
from os.path import join
import yaml

for i, dir in enumerate(tqdm(decayData_dir)):
    loc = locs[i]
    roshan_dir = f"{dir}/roshan_{loc}"
    if not exists(roshan_dir):
        mkdir(roshan_dir)
    with open(join(dir, "..", "SSMmodel", "pre-tensioned_rest_q.pkl"), "rb") as f:
        rest_q = pickle.load(f)
    with open(join(dir, "info.yaml"), "r") as f:
        pre_tensioning = yaml.safe_load(f)['pre_tensioning']
    # save rest_q array to csv
    np.savetxt(f"{roshan_dir}/rest_pos_{loc}.csv", rest_q, fmt='%.5e')
    np.savetxt(f"{roshan_dir}/pre_tensioning_u_{loc}.csv", pre_tensioning, fmt='%.5e')

  0%|          | 0/9 [00:00<?, ?it/s]

In [15]:
import tarfile
import os.path

out_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/data_roshan_trunk_adiabatic"
if not exists(out_dir):
    mkdir(out_dir)
for i, dir in enumerate(tqdm(decayData_dir)):
    loc = locs[i]
    roshan_dir = f"{dir}/roshan_{loc}"
    with tarfile.open(join(out_dir, f"{loc}.tar.gz"), "w:gz") as tar:
        tar.add(roshan_dir, arcname=os.path.basename(roshan_dir))

  0%|          | 0/9 [00:00<?, ?it/s]